In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
df=input_data.read_data_sets('tmp/data/',one_hot=True)

#neural network parameter
num_input=784
num_class=10
dropout=.6
x=tf.placeholder(tf.float32,shape=[None,num_input])
y=tf.placeholder(tf.float32,shape=[None,num_class])
keep_prob=tf.placeholder(tf.float32)
batch_size=200
num_step=400
learning_rate=1e-3
display_Step=20

#creating network architecture
def convolution_img(x,W,b,strides=1):
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding='SAME')
    x=tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def max_pooling(x,k=3):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')

#weight and bias
W={"w1":tf.Variable(tf.random_normal([5,5,1,32])),
   "w2":tf.Variable(tf.random_normal([5,5,32,64])),
   "w3":tf.Variable(tf.random_normal([7*7*64,1024])),
   "w4":tf.Variable(tf.random_normal([1024,num_class]))}

b={"b1":tf.Variable(tf.random_normal([32])),
   "b2":tf.Variable(tf.random_normal([64])),
   "b3":tf.Variable(tf.random_normal([1024])),
   "b4":tf.Variable(tf.random_normal([num_class]))}

def model_conv(x,W,b,dropout):
    x=tf.reshape(x,shape=[-1,28,28,1])
    
    #convolution and maxpool layer
    
    c1=convolution_img(x,W["w1"],b["b1"],strides=1)
    c1=max_pooling(c1,k=2)
    c1=convolution_img(c1,W["w2"],b["b2"],strides=1)
    c1=max_pooling(c1,k=2)
    
    #fullyconnected layer
    
    c1=tf.reshape(c1,[-1,W["w3"].get_shape().as_list()[0]])
    c1=tf.matmul(c1,W['w3'])
    c1=tf.add(c1,b["b3"])
    c1=tf.nn.relu(c1)
    
    #dropout
    
    c1=tf.nn.dropout(c1,dropout)
    
    #output
    c1=tf.matmul(c1,W['w4'])
    c1=tf.add(c1,b["b4"])
    
    return c1


probs=model_conv(x,W,b,dropout=keep_prob)
predicted=tf.nn.softmax(probs)

cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=probs , labels=y)

cost=tf.reduce_mean(cross_entropy)

optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)
actual=tf.argmax(y,1)
preds=tf.argmax(predicted,1)

accuracy=tf.equal(actual,preds)
accuracy=tf.cast(accuracy,tf.float32)
accuracy=tf.reduce_mean(accuracy)

#running the tensorflow

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for step in range(1,num_step +1):
        batch_x,batch_y=df.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:dropout})
        if step % 20  == 0 or step==1 :
            loss,acc=sess.run([cost,accuracy],feed_dict={x:batch_x,y:batch_y,keep_prob:1.0})
            print("step-%i : loss=%f : training_accuracy=%f" %(step,loss,acc))
            #print("training_Accuracy-%f:"%(acc))
    print('optimization finished')
    
    #calculate the accuracy for test data set
    print("testing accuracy:", sess.run(accuracy,feed_dict={x:df.test.images[:256], y:df.test.labels[:256], keep_prob:1.0}))
                                                            
            




    
    
    






Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on back